This code was developed based on Google Colab

In [ ]:
## Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance

import re
import seaborn as sns

from scipy.stats import linregress
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor
from econml.dml import CausalForestDML

In [ ]:
## Read files and data clean
def read_satellite_data(satellite):
    # --- Main Execution ---
    df_satellite = pd.read_csv(f'/content/drive/My Drive/Colab Notebooks/Phenology/Satellite Code/data/tables/phenology_climate/{satellite}.csv')
    # print(df_satellite)

    veg_class = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Phenology/Satellite Code/data/tables/veg_class/veg_class.csv')
    df = pd.merge(df_satellite, veg_class, on=['latitude', 'longitude'], how='inner')
    df = df[df['veg_class'].isin([11, 12, 13, 14])]
    return df

In [ ]:
def causal_ml(data, name, mv):
    feature_names = get_feature_names()
    years, windows = get_years_and_windows(name, mv)
    # print(windows)
    # print(years)
    all_effects = {}  # key: (lat, lon), value: dict of effects

    for i, (start, end) in enumerate(windows):
        print(f'Processing window {i + 1}')
        window_label = f"_es_w{i+1}"
        # middle_year = (start + end) // 2
        # window_label = f"_es_{middle_year}"

        for idx, row in data.iterrows():
            key = (row['latitude'], row['longitude'])

            # Compute effect for this window
            effect, pvalue = model_pixel_window(row, start, end, years, feature_names, name)

            # Initialize row dict if first time
            if key not in all_effects:
                all_effects[key] = {
                    'latitude': row['latitude'],
                    'longitude': row['longitude']
                }

            # Add each feature's effect to the row
            for fname, val, pval in zip(feature_names, effect, pvalue):
            # for fname, val in zip(['sos', 'summer_t', 'autumn_t'], effect):
                # print(fname)
                all_effects[key][f"{fname}{window_label}"] = val
                all_effects[key][f"{fname}_es_p_w{i+1}"] = pval

                # print('===============')
                # print(all_effects)
    # Convert to DataFrame
    df_effects = pd.DataFrame(list(all_effects.values()))
    return df_effects


In [ ]:
from sklearn.linear_model import LinearRegression
from econml.dml import LinearDML

## Feature names
def get_feature_names():
    return ['sos', 'spring_t', 'summer_t', 'autumn_t',
            'spring_p', 'summer_p', 'autumn_p',
            'spring_r', 'summer_r', 'autumn_r', ]
## Modeling a single row-window
def model_pixel_window(row, start_year, end_year, years, feature_names, name):
    window_years = [y for y in years if start_year <= y <= end_year]

    df_long = pd.DataFrame({
        'year': window_years,
        'eos': [row[f'eos_{y}'] for y in window_years],
        'sos': [row[f'sos_{y}'] for y in window_years],
        **{f: [row[f'{f}_{y}'] for y in window_years] for f in feature_names if f != 'sos'}
    })
    # print(df_long)
    # PEP725 special filtering
    if name == 'pep725':
        df_long = df_long[df_long['eos'].notna() & df_long['sos'].notna()]
        if len(df_long) < 10:
            return np.full(len(feature_names), np.nan)
    # print(df_long)
    # Standardization
    for col in feature_names + ['eos']:
        # mean, std = df_long[col].mean(), df_long[col].std()
        # df_long[col] = (df_long[col] - mean) / std if std > 0 else 0.0
        min_val = df_long[col].min()
        max_val = df_long[col].max()
        if max_val > min_val:
            df_long[col] = 2 * (df_long[col] - min_val) / (max_val - min_val) - 1
        else:
            df_long[col] = 0.0  # Constant column

    # return cf.const_marginal_effect(X).mean(axis=0)
    # Loop over each feature as treatment
    effect_estimates = []
    pvalue_estimates = []
    for i, treat_var in enumerate(feature_names):
        test_var = True
        # if treat_var == "sos" or treat_var == "summer_t" or treat_var == "autumn_t":
        # if treat_var == "sos" or treat_var == "summer_t" or treat_var == "autumn_t" or treat_var == "spring_t" or treat_var == "spring_p" or treat_var == "summer_p" or treat_var == "autumn_p":
        if test_var == True:
            T = df_long[treat_var].to_numpy().ravel()
            Y = df_long['eos'].to_numpy().ravel()
            control_vars = [f for f in feature_names if f != treat_var]
            W = df_long[control_vars].to_numpy() if control_vars else np.ones((len(Y), 1))

            cf = LinearDML(
                model_y=LinearRegression(),  # OLS for Y
                model_t=LinearRegression(),  # OLS for T
                discrete_treatment=False,
                random_state=42
            )


            try:
                cf.fit(Y, T, W=W)
                effect = cf.ate()
                ate_inf = cf.ate_inference()
                p_value = ate_inf.pvalue()

            except Exception as e:
                print(f"Error for {treat_var}: {e}")
                effect = np.nan
                p_value = np.nan

            effect_estimates.append(effect)
            pvalue_estimates.append(p_value)

    return np.array(effect_estimates), np.array(pvalue_estimates)

In [ ]:
## Years & moving windows
def get_years_and_windows(name, mv):
    if name == 'avhrr':
        years = list(range(1982, 2017))
        windows = [(1982 + i, 1998 + i) for i in range(19)] if mv else [(1982, 2017)]
    elif name == 'avhrr_1982_1999':
        years = list(range(1982, 2000))
        windows = [(1982, 2000)]
    elif name == 'avhrr_2000_2016':
        years = list(range(2000, 2017))
        windows = [(2000, 2017)]
    elif name == 'modis':
        years = list(range(2001, 2024))
        windows = [(2001 + i, 2015 + i) for i in range(9)] if mv else [(2001, 2024)]
    elif name == 'pep725':
        years = list(range(1951, 2016)) if not mv else list(range(1951, 2016))
        windows = (
            [(1951, 2016)] if not mv else
            [(1951, 1964), (1953, 1966), (1955, 1969), (1957, 1971),
             (1960, 1974), (1963, 1977), (1965, 1979), (1968, 1981),
             (1970, 1984), (1973, 1987), (1975, 1989), (1977, 1991),
             (1980, 1994), (1983, 1997), (1985, 1999), (1988, 2001),
             (1990, 2004), (1993, 2007), (1995, 2009), (1997, 2012),
             (2000, 2016)]
        )
    # print(years)
    # print(windows)
    return years, windows

In [ ]:
## Read avhrr data
satellite = "avhrr"
df = read_satellite_data(satellite)
print(df.shape)
# df = df.sample(frac=1, random_state=42).reset_index(drop=True).iloc[0:10]


(128264, 458)


In [ ]:
# sampled_df = df.sample(frac=1, random_state=42).reset_index(drop=True).iloc[0:20]
sampled_df = df

In [ ]:
## Causal effect size using all-year data
import time
start_time = time.time()
mv = False
window_effect = causal_ml(sampled_df, satellite, mv)
window_effect.to_csv("/content/drive/My Drive/Colab Notebooks/Phenology/Figures Code/results/figure3/effect_size/avhrr/effect_size_ate.csv", index=False)
print(window_effect.head())
end_time = time.time()
print(f"Time taken: {end_time - start_time:.4f} seconds")

Processing window 1
   latitude  longitude  sos_es_w1  sos_es_p_w1  spring_t_es_w1  \
0    70.355     82.455  -0.243237     0.012673        0.762881   
1    70.155    125.855   0.011572     0.959165        0.089543   
2    70.055     82.455  -0.260354     0.190741        0.002323   
3    70.055    125.355   0.381705     0.143653        0.123715   
4    69.955     82.955   0.535225     0.090200        0.309039   

   spring_t_es_p_w1  summer_t_es_w1  summer_t_es_p_w1  autumn_t_es_w1  \
0          0.131538       -0.637895          0.111362       -0.001629   
1          0.715210        0.194506          0.215534        0.144848   
2          0.993553       -0.863755          0.001055       -0.487011   
3          0.715914        0.031318          0.910296       -0.203216   
4          0.372307       -0.344558          0.376429       -0.285935   

   autumn_t_es_p_w1  ...  summer_p_es_w1  summer_p_es_p_w1  autumn_p_es_w1  \
0          0.989851  ...        0.230891          0.263825        

In [ ]:
## Causal effect size of two periods
sampled_df = df#.iloc[0:10]
import time
start_time = time.time()
mv = False
year_cols = [col for col in sampled_df.columns if "_" in col and col.split("_")[-1].isdigit()]

def get_year_cols(cols, start, end):
    return [col for col in cols if start <= int(col.split("_")[-1]) <= end]
cols_82_99 = get_year_cols(year_cols, 1982, 1999)
cols_00_16 = get_year_cols(year_cols, 2000, 2016)
df_82_99 = sampled_df[["longitude", "latitude", "veg_class"] + cols_82_99]
df_00_16 = sampled_df[["longitude", "latitude", "veg_class"] + cols_00_16]

window_effect = causal_ml(df_82_99, 'avhrr_1982_1999', mv)
window_effect.to_csv("/content/drive/My Drive/Colab Notebooks/Phenology/Figures Code/results/figure3/effect_size/avhrr/effect_size_82_99_ate.csv", index=False)

print('========================')
window_effect = causal_ml(df_00_16, 'avhrr_2000_2016', mv)
window_effect.to_csv("/content/drive/My Drive/Colab Notebooks/Phenology/Figures Code/results/figure3/effect_size/avhrr/effect_size_00_16_ate.csv", index=False)

# print(window_effect.head())
end_time = time.time()
print(f"Time taken: {end_time - start_time:.4f} seconds")

Processing window 1
Processing window 1
Time taken: 39875.5233 seconds
